In [1]:
from sklearn.externals import joblib
import pickle
import sklearn 
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import train_test_split
import numpy as np

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [76]:
X= joblib.load("smote_scaled_x.pkl")
y= joblib.load("smote_scaled_y.pkl")

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.1, random_state=42)

### Random Forest

In [78]:

rf= RandomForestClassifier()

In [79]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [80]:
predictions=rf.predict(X_test)

In [81]:

tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
(tn, fp, fn, tp)

(28, 4, 4, 31)

In [83]:
accuracy_score(y_test, predictions)

0.8805970149253731

In [203]:
#Tuning the model using CV grid

param_grid = {"n_estimators": [25,50,75,100,125,150], "criterion":('gini','entropy')}
grid_cv= GridSearchCV(rf, param_grid)
grid_cv.fit(X_train, y_train)
predictions_grid_rf=grid_cv.predict(X_test)
print(accuracy_score(y_test, predictions_grid_rf))
tn, fp, fn, tp=confusion_matrix(y_test,predictions_grid_rf).ravel()
(tn, fp, fn, tp)


0.8955223880597015


(28, 4, 3, 32)

In [199]:
grid_cv.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

### Logistic Regression 

In [84]:
lg= LogisticRegression()

In [85]:
lg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [86]:
predictions_lg=lg.predict(X_test)

In [95]:
tn, fp, fn, tp = confusion_matrix(y_test, predictions_lg).ravel()
(tn, fp, fn, tp)

(23, 9, 1, 34)

In [88]:
accuracy_score(y_test, predictions_lg)

0.8507462686567164

### Support Vector Classifier

In [184]:
svc= SVC()

In [201]:
svc.fit(X_train, y_train)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [202]:
predictions_svc=svc.predict(X_test)

In [172]:
tn, fp, fn, tp=confusion_matrix(y_test, predictions_svc).ravel()
(tn, fp, fn, tp)


(22, 10, 1, 34)

In [117]:
accuracy_score(y_test, predictions_svc)

0.835820895522388

In [190]:
#Hyperparameter tuning
parameters = {"kernel":('rbf', 'sigmoid', 'poly') , "C": [1,5,10,15]}
grid=GridSearchCV(svc, parameters)
grid.fit(X_train, y_train)
predictions_svct=grid.predict(X_test)
grid.best_estimator_

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [193]:
print(accuracy_score(y_test, predictions_svct))

tn, fp, fn, tp=confusion_matrix(y_test, predictions_svct).ravel()
(tn, fp, fn, tp)

0.8955223880597015


(26, 6, 1, 34)

### Neural Network (KERAS)

In [119]:
!pip install keras

You should consider upgrading via the 'pip install --upgrade pip' command.


In [122]:
from keras import layers
from keras import Sequential 
from keras.layers import Dense

In [167]:
classifier=Sequential()
classifier.add(Dense(200,activation='relu',kernel_initializer='random_normal', input_dim=X_test.shape[1]))
classifier.add(Dense(400,activation='relu',kernel_initializer='random_normal'))
classifier.add(Dense(400,activation='relu',kernel_initializer='random_normal'))
#classifier.add(Dense(20,activation='relu',kernel_initializer='random_normal'))
#classifier.add(layers.Dropout(0.05))
classifier.add(Dense(1,activation='sigmoid',kernel_initializer='random_normal'))
classifier.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
#classifier.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])
classifier.fit(X_train, y_train, batch_size=20, verbose=0, epochs=100) #epochs=100
eval_model=classifier.evaluate(X_test, y_test)
eval_model

67/67 [==============================] - 1s 8ms/step


[0.5323366469411708, 0.8955223880597015]

In [165]:
predictions_nn=classifier.predict(X_test)
predictions_nn=(predictions_nn >0.50)
tn, fp, fn, tp=confusion_matrix(y_test, predictions_nn).ravel()
(tn, fp, fn, tp)

(24, 8, 2, 33)

In [166]:
accuracy_score(y_test, predictions_nb)

0.6567164179104478

### Naive Bayes

In [266]:
nb= ComplementNB()
nb.fit(X_train, y_train)

ComplementNB(alpha=1.0, class_prior=None, fit_prior=True, norm=False)

In [267]:
predictions_nb=nb.predict(X_test)

In [268]:
tn, fp, fn, tp=confusion_matrix(y_test, predictions_nb).ravel()
(tn, fp, fn, tp)

(21, 11, 12, 23)

In [269]:
accuracy_score(y_test, predictions_nb)

0.6567164179104478

In [271]:
#Hyperparameter optmization

nb_tuned= ComplementNB(alpha=0)

nb_tuned.fit(X_train, y_train)

predictions_nbt=nb_tuned.predict(X_test)

accuracy_score(y_test, predictions_nbt)

tn, fp, fn, tp=confusion_matrix(y_test, predictions_nbt).ravel()
fp

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


11

## Model Evaluation

In [259]:
model_list =[
predictions,
predictions_grid_rf,
predictions_nn,
predictions_svc,
predictions_svct,
predictions_nb,
predictions_nbt,

]

In [280]:
#Create a list of accuracy scores for each model
model_eval= []
for (index,i) in enumerate (model_list):
    score=accuracy_score(y_test, i)
    model_eval.insert(index,score)
#Create a list of false positive rate for each model
model_eval_fp= []
for i in model_list:
    tn, fp, fn, tp=confusion_matrix(y_test, i).ravel()
    model_eval_fp.append(fp)
model_eval_fp

[4, 4, 8, 10, 6, 11, 11]

In [261]:
model_eval

[0.8805970149253731,
 0.8955223880597015,
 0.8507462686567164,
 0.835820895522388,
 0.8955223880597015,
 0.6567164179104478,
 0.6567164179104478]

In [262]:
model_names = ['Random Forest', 'Random Forest Tuned', 'Neural Network', 
               'Support Vector Classifier', 'Support Vector Classifier Tuned', 
              'Naive Bayes', 'Naive Bayes Tuned']

In [263]:
model_names


['Random Forest',
 'Random Forest Tuned',
 'Neural Network',
 'Support Vector Classifier',
 'Support Vector Classifier Tuned',
 'Naive Bayes',
 'Naive Bayes Tuned']

In [281]:
model_eval_df=pd.DataFrame([model_eval_fp, model_eval], columns=model_names).transpose()

In [283]:
model_eval_df.columns= ['False Positive Rate','Accuracy Score']
model_eval_df.sort_values(['False Positive Rate'])

,False Positive Rate,Accuracy Score
Random Forest,4.0,0.880597
Random Forest Tuned,4.0,0.895522
Support Vector Classifier Tuned,6.0,0.895522
Neural Network,8.0,0.850746
Support Vector Classifier,10.0,0.835821
Naive Bayes,11.0,0.656716
Naive Bayes Tuned,11.0,0.656716


### Instntiate the model with the necessary parameters and pickle it

In [284]:
final_model = RandomForestClassifier(n_estimators=100, criterion='entropy')

In [285]:
final_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [286]:
filename= 'final_model.pkl'
joblib.dump(final_model, filename)

['final_model.pkl']

In [288]:
!pip freeze

absl-py==0.9.0
alabaster==0.7.10
anaconda-client==1.6.14
anaconda-project==0.8.2
asn1crypto==0.24.0
astor==0.8.1
astroid==1.6.3
astropy==3.0.2
attrs==18.1.0
Automat==0.3.0
autovizwidget==0.15.0
awscli==1.18.47
Babel==2.5.3
backcall==0.1.0
backports.shutil-get-terminal-size==1.0.0
bcrypt==3.1.7
beautifulsoup4==4.6.0
bitarray==0.8.1
bkcharts==0.2
blaze==0.11.3
bleach==2.1.3
bokeh==1.4.0
boto==2.48.0
boto3==1.12.47
botocore==1.15.47
Bottleneck==1.2.1
cached-property==1.5.1
certifi==2019.11.28
cffi==1.11.5
characteristic==14.3.0
chardet==3.0.4
click==6.7
cloudpickle==0.5.3
clyent==1.2.2
colorama==0.3.9
contextlib2==0.5.5
cryptography==2.9.2
cycler==0.10.0
Cython==0.28.4
cytoolz==0.9.0.1
dask==1.2.2
datashape==0.5.4
decorator==4.3.0
defusedxml==0.6.0
distributed==1.28.1
docker==4.2.0
docker-compose==1.25.5
dockerpty==0.4.1
docopt==0.6.2
docutils==0.14
entrypoints==0.2.3
enum34==1.1.9
environment-kernels==1.1.1
et-xmlfile==1.0.1
fastcache==1.0.2
filelock==3.0.4
Flask==1.0.2
Flask-Cors==3.0.4